In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [2]:
#input: city or airport

#return: airline

In [3]:
headers = {
    "X-RapidAPI-Key": "9062d689c2mshe1b9e4c756a6482p165546jsn847c6c6ea1cc",
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }


def get_codes():
    
    location = input('Insert location:')
    print()
    
    url = "https://aerodatabox.p.rapidapi.com/airports/search/term"
    querystring = {"q":location.lower(), "limit":"10"}

    response = requests.get(url, headers=headers, params=querystring)
    codes = response.json()['items'] if 'items' in response.json() else location

    return codes


def choose_code():
    
    codes = get_codes()
    
    while type(codes) is str or len(set([code['shortName'] for code in codes])) != 1:
        
        if not codes:
            print('Try another location. Unable to find code for:', codes, '\n')
        else:
            print('More than one location found. Insert one of the following airports:', [code['shortName'] for code in codes], '\n')
        
        codes = get_codes()
    
    return codes[0]['icao']


def flight_between(from_, to_):
    url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{from_}/stats/routes/daily"
    response = requests.get(url, headers=headers)
    resp_json = response.json()

    results_list = []
    routes = resp_json['routes']
    for i in range(0, len(routes)):
        route = routes[i]
        if 'icao' in route['destination'] and route['destination']['icao'] == to_:
            results_list.extend(route['operators'])


   
    json_list = json.dumps(results_list)
    return pd.json_normalize(results_list).T

In [5]:
print('[DEPARTURE]')
from_ = choose_code()
print('[DESTINATION]')
to_ = choose_code()

flight_df = flight_between(from_, to_)
flight_df

[DEPARTURE]
Insert location:Paris

More than one location found. Insert one of the following airports: ['Charles de Gaulle', '-Orly', "Le Touquet-Côte d'Opale", 'Paris Beauvais Tillé', '-Le Bourget', 'Henry County'] 

Insert location:Charles de Gaulle

[DESTINATION]
Insert location:Alger



,0,1,2
name,Air Algerie,Air France,ASL France


In [ ]:
#inputs: airport, departure and arrival date
#returns: delay

In [13]:
import requests


location = input('Please enter the exact icao code:').upper()
from_when = input('Please enter the time in this format:\n \"YYYY-MM-DDThh:mm"')
to_when = input('Please enter the time in this format:\n \"YYYY-MM-DDThh:mm"')

url = f"https://aerodatabox.p.rapidapi.com/airports/icao/{location}/delays/{from_when}/{to_when}"

headers = {
    "X-RapidAPI-Key": "9062d689c2mshe1b9e4c756a6482p165546jsn847c6c6ea1cc",
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)
print(response)
print(response.text)

Please enter the exact icao code:EHAM
Please enter the time in this format:
 "YYYY-MM-DDThh:mm"2022-05-06T12:00
Please enter the time in this format:
 "YYYY-MM-DDThh:mm"2022-05-07T00:00
<Response [200]>
[{"airportIcao":"EHAM","fromLocal":"2022-05-06 10:00+02:00","toLocal":"2022-05-06 12:00+02:00","fromUtc":"2022-05-06 08:00Z","toUtc":"2022-05-06 10:00Z","departuresDelayInformation":{"numTotal":97,"numQualifiedTotal":89,"numCancelled":2,"medianDelay":"00:20:00","delayIndex":1.17},"arrivalsDelayInformation":{"numTotal":98,"numQualifiedTotal":88,"numCancelled":2,"medianDelay":"-00:07:00","delayIndex":0.0}},{"airportIcao":"EHAM","fromLocal":"2022-05-06 10:15+02:00","toLocal":"2022-05-06 12:15+02:00","fromUtc":"2022-05-06 08:15Z","toUtc":"2022-05-06 10:15Z","departuresDelayInformation":{"numTotal":95,"numQualifiedTotal":88,"numCancelled":2,"medianDelay":"00:19:00","delayIndex":1.14},"arrivalsDelayInformation":{"numTotal":93,"numQualifiedTotal":82,"numCancelled":2,"medianDelay":"-00:07:00","

In [14]:
pd.json_normalize(response.json())

,airportIcao,fromLocal,toLocal,fromUtc,toUtc,departuresDelayInformation.numTotal,departuresDelayInformation.numQualifiedTotal,departuresDelayInformation.numCancelled,departuresDelayInformation.medianDelay,departuresDelayInformation.delayIndex,arrivalsDelayInformation.numTotal,arrivalsDelayInformation.numQualifiedTotal,arrivalsDelayInformation.numCancelled,arrivalsDelayInformation.medianDelay,arrivalsDelayInformation.delayIndex
0,EHAM,2022-05-06 10:00+02:00,2022-05-06 12:00+02:00,2022-05-06 08:00Z,2022-05-06 10:00Z,97,89,2,00:20:00,1.17,98,88,2,-00:07:00,0.00
1,EHAM,2022-05-06 10:15+02:00,2022-05-06 12:15+02:00,2022-05-06 08:15Z,2022-05-06 10:15Z,95,88,2,00:19:00,1.14,93,82,2,-00:07:00,0.00
2,EHAM,2022-05-06 10:30+02:00,2022-05-06 12:30+02:00,2022-05-06 08:30Z,2022-05-06 10:30Z,99,91,1,00:19:00,1.06,91,81,2,-00:07:00,0.00
3,EHAM,2022-05-06 10:45+02:00,2022-05-06 12:45+02:00,2022-05-06 08:45Z,2022-05-06 10:45Z,105,94,1,00:18:00,1.00,89,79,2,-00:08:00,0.00
4,EHAM,2022-05-06 11:00+02:00,2022-05-06 13:00+02:00,2022-05-06 09:00Z,2022-05-06 11:00Z,101,90,1,00:19:00,1.06,85,74,2,-00:09:00,0.00
5,EHAM,2022-05-06 11:15+02:00,2022-05-06 13:15+02:00,2022-05-06 09:15Z,2022-05-06 11:15Z,100,90,1,00:19:00,1.06,83,73,1,-00:08:00,0.00
6,EHAM,2022-05-06 11:30+02:00,2022-05-06 13:30+02:00,2022-05-06 09:30Z,2022-05-06 11:30Z,104,92,1,00:18:00,1.00,80,72,0,-00:07:00,0.00
7,EHAM,2022-05-06 11:45+02:00,2022-05-06 13:45+02:00,2022-05-06 09:45Z,2022-05-06 11:45Z,105,92,1,00:16:00,0.92,79,72,0,-00:06:00,0.00
8,EHAM,2022-05-06 12:00+02:00,2022-05-06 14:00+02:00,2022-05-06 10:00Z,2022-05-06 12:00Z,101,88,1,00:16:00,0.89,81,74,0,-00:04:00,0.00
9,EHAM,2022-05-06 12:15+02:00,2022-05-06 14:15+02:00,2022-05-06 10:15Z,2022-05-06 12:15Z,101,89,1,00:13:00,0.72,89,82,1,-00:02:00,0.00
